In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from allensdk.api.queries.ontologies_api import OntologiesApi
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import cm
import operator

import statsmodels.api as sm

sns.set_context('poster')
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque'

In [ ]:
dat = pd.read_csv(os.path.join(path, 'unionizes', 'experiment_structure_unionizes_consolidated_06_10_2018.csv'))

In [ ]:
dat.head()

In [ ]:
count = pd.read_csv(r'C:\Users\jenniferwh\Dropbox (Allen Institute)\Mesoscale Connectome Papers in Progress\2018 Plaque\plaque counts\plaque_counts.csv')

In [ ]:
meta = pd.read_csv(os.path.join(path, 'plaque_dataset_07_06_2018.csv'))

In [ ]:
meta.head(1)

In [ ]:
meta.keys()

In [ ]:
meta['Plaque map call (R hemisphere)'].unique()

In [ ]:
dataset = meta[meta['Plaque map call (R hemisphere)'] == 'Pass']

In [ ]:
len(dataset)

In [ ]:
oapi = OntologiesApi()
summary_structures = oapi.get_structures(structure_set_names="'Mouse Connectivity - Summary'")
summary_structure_ids = [item['id'] for item in summary_structures]
summary_structure_ids.remove(934)
print(len(summary_structure_ids))

In [ ]:
coarse_structures = oapi.get_structures(structure_set_names="'Mouse - Coarse'")
coarse_structure_ids = [item['id'] for item in coarse_structures]
print(len(coarse_structure_ids))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.iteritems()}

In [ ]:
ai_map[997]

In [ ]:
structures = summary_structure_ids + coarse_structure_ids + [997]

In [ ]:
dataset_ids = dat['image_series_id'].unique()
print(len(dataset_ids))

In [ ]:
isids = [int(value) for value in dataset['Link to image series']]

In [ ]:
dataset['image_series_id'] = isids

In [ ]:
dataset = dataset[dataset['image_series_id'].isin(dataset_ids)]
print(len(dataset))

In [ ]:
dataset.keys()

In [ ]:
dataset.Experiment.unique()

In [ ]:
dataset.loc[dataset['Specimen ID'].str.contains('APP/PS1'), 'Mouse Line'] = 'APP/PS1'
dataset.loc[dataset['Specimen ID'].str.contains('APP_PS1'), 'Mouse Line'] = 'APP/PS1'
dataset.loc[dataset['Specimen ID'].str.contains('J20'), 'Mouse Line'] = 'J20'
dataset.loc[dataset['Specimen ID'].str.contains('Tg2576'), 'Mouse Line'] = 'Tg2576'
dataset.loc[dataset['Specimen ID'].str.contains('rTg4510'), 'Mouse Line'] = 'APP/PS1/rTg4510'
dataset.loc[dataset['Specimen ID'].str.contains('Cre'), 'Mouse Line'] = 'APP/PS1/Cre'

In [ ]:
dataset[dataset['Mouse Line'].isnull()]['Specimen ID']

In [ ]:
dataset['Mouse Line'].unique()

In [ ]:
dataset.groupby(['Mouse Line', 'age group']).count()['Unnamed: 0']

In [ ]:
dataset = dataset[dataset['Mouse Line'] != 'APP/PS1/Cre']

In [ ]:
dataset.groupby(['Mouse Line', 'age group']).count()['Unnamed: 0']

In [ ]:
dat.head()

In [ ]:
count.head()

In [ ]:
dat = dat[['image_series_id', 'structure_id', 'plaque_density']].merge(
    count[['image_series_id', 'structure_id', 'sum_plaques']], on=['image_series_id', 'structure_id'])

In [ ]:
dat.head()

In [ ]:
dat['structure_id'].unique()

In [ ]:
dat = dat[~dat['structure_id'].isin([526322264, 563807439, 589508451, 1060])] #these structures are throwing errors in OLS

In [ ]:
pltdat = dat[dat['structure_id'] == 315]
fig, ax = plt.subplots()
ax.scatter(pltdat['sum_plaques'], pltdat['plaque_density'])

In [ ]:
dat.head()

In [ ]:
def correlate_density_count(image_series_ids):
    subset_dat = dat[dat['image_series_id'].isin(image_series_ids)]
    structure_name = []
    structure_id = []
    rsquared = []
    pvals = []
    pvals_x = []
    pvals_constant = []
    for structure in subset_dat['structure_id'].unique():
        calcdat = subset_dat[subset_dat['structure_id'] == structure]
        y = calcdat['plaque_density']
        X = calcdat['sum_plaques']
        X = sm.add_constant(X)
        res = sm.OLS(y, X).fit()
        try:
            if res.pvalues['const'] > 0:
                structure_id.append(structure)
                structure_name.append(ai_map[structure])
                rsquared.append(res.rsquared_adj)
                pvals.append(res.pvalues)
                pvals_x.append(res.pvalues['sum_plaques'])
                pvals_constant.append(res.pvalues['const'])
        except:
            continue
    fdrcorr_x = sm.stats.fdrcorrection(pvals_x, alpha=0.05, method='indep')
    fdrcorr_const = sm.stats.fdrcorrection(pvals_constant, alpha=0.05, method='indep')
    corrdat = pd.DataFrame({'structure_name': structure_name, 'structure_id': structure_id,
                            'r_squared': rsquared, 'p_value': fdrcorr_x[1],
                           'p_value_constant': pvals_constant})
    return corrdat

In [ ]:
all_corrdat = correlate_density_count(dat['image_series_id'])

In [ ]:
all_corrdat.sort_values(by='p_value')

In [ ]:
no_corr_strs = all_corrdat[all_corrdat['p_value'] > 0.05]['structure_id'].values

In [ ]:
no_corr_strs

In [ ]:
dat[dat['structure_id'].isin(no_corr_strs)].describe()

In [ ]:
dat[~dat['structure_id'].isin(no_corr_strs)].describe()

In [ ]:
pltdat = dat[dat['structure_id'] == 382]
fig, ax = plt.subplots()
ax.scatter(pltdat['sum_plaques'], pltdat['plaque_density'])

In [ ]:
APP_PS1 = map(int, dataset[dataset['Mouse Line'] == 'APP/PS1']['Link to image series'])
APP_PS1_corrdat = correlate_density_count(APP_PS1)

In [ ]:
APP_PS1_corrdat.sort_values(by='p_value')

In [ ]:
APP_PS1_corrdat.mean()

In [ ]:
all_corrdat.mean()

In [ ]:
dataset['Mouse Line'].unique()

In [ ]:
J20 = map(int, dataset[dataset['Mouse Line'] == 'J20']['Link to image series'])
J20_corrdat = correlate_density_count(J20)
J20_corrdat.mean()

In [ ]:
triple = map(int, dataset[dataset['Mouse Line'] == 'APP/PS1/rTg4510']['Link to image series'])
triple_corrdat = correlate_density_count(triple)
triple_corrdat.mean()

In [ ]:
Tg2576 = map(int, dataset[dataset['Mouse Line'] == 'Tg2576']['Link to image series'])
Tg2576_corrdat = correlate_density_count(Tg2576)
Tg2576_corrdat.mean()

In [ ]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
hipp = structure_tree.get_structures_by_acronym(['HPF'])[0]
ptlp = structure_tree.get_structures_by_acronym(['PTLp'])[0]
isohipp_ids = structure_tree.descendant_ids([iso['id'], hipp['id'], ptlp['id']])
print(len(isohipp_ids))
isohipp_ids = [y for x in isohipp_ids for y in x]
print(len(isohipp_ids))

In [ ]:
isohipp_ids = [ids for ids in isohipp_ids if ids in summary_structure_ids]
print(len(isohipp_ids))

## Generate dictionary with p value per structure to see where correlation is bad

In [ ]:
all_corrdat.head()

In [ ]:
structure_vals = dict(zip(all_corrdat['structure_id'], all_corrdat['p_value']))

In [ ]:
structure_vals[315]

In [ ]:
mask, _ = mcc.get_structure_mask(997)
print(mask.shape)
plt.imshow(mask[200])
print(np.unique(mask))

In [ ]:
coarse_structure_ids

In [ ]:
mask, _ = mcc.get_structure_mask(997)
newmask = np.zeros(mask.shape)
for structure in coarse_structure_ids:
    structure_mask, _ = mcc.get_structure_mask(structure)
    newval = structure_vals[structure]
    newmask[np.where(structure_mask)] = newval
print(np.unique(newmask))
plt.imshow(newmask[300])

In [ ]:
plt.imshow(newmask[:,200,:])

In [ ]:
plt.imshow(newmask[:,:,100])

In [ ]:
newmask.shape

In [ ]:
np.unique(newmask)

In [ ]:
# This takes forever to run
mask, _ = mcc.get_structure_mask(997)
newmask = np.zeros_like(mask)
for structure in summary_structure_ids:
    structure_mask, _ = mcc.get_structure_mask(structure)
    newval = structure_vals[structure]*10000
    newmask[np.where(structure_mask)] = newval
print(np.unique(newmask))
plt.imshow(newmask[300])

In [ ]:
plt.imshow(newmask[:,155,:])

In [ ]:
plt.imshow(newmask[:,:,250])

In [ ]:
from matplotlib import cm
structure_rgb_vals = structure_vals.copy()
for key in structure_rgb_vals:
    structure_rgb_vals[key] = tuple([255*i for i in cm.inferno(structure_vals[key]*50)[:3]])
structure_rgb_vals[0] = [0.0, 0.0, 0.0]

In [ ]:
reference_space =  mcc.get_reference_space()
slice_image = reference_space.get_slice_image(1, 6000, structure_rgb_vals) # this method wants an axis and a position in microns
plt.imshow(slice_image)